In [1]:
from scipy.io import arff
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import sklearn.preprocessing as skpre
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import confusion_matrix
from functools import reduce
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_wine
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_predict
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import  make_scorer
from sklearn import metrics

C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
D:\Anaconda\lib\site-packages\sklearn\linear_model\least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the nump

In [3]:
warnings.filterwarnings('ignore')
def Dummy(yields,a=.02,b=-.02):  #change raise to state
    dummy=[]
    for i in range(len(yields)):
        if yields.iloc[i]>=a:
            dummy.append(1)
        elif yields.iloc[i]<b:
            dummy.append(-1)
        else:
            dummy.append(0)
    output=pd.DataFrame(dummy)
    return output
def metrics(C):   #output rates
    print('TrueShortRate=',C[0][0]/C[0].sum())
    print('FalseShortRate=',C[0][2]/C[0].sum())
    print('CaughtShortRate=',C[0][0]/C.iloc[0,:].sum())
    print('TrueLongRate=',C[2][2]/C[2].sum())
    print('FalseLongRate=',C[2][0]/C[2].sum())
    print('CaughtLongRate=',C[2][2]/C.iloc[2,:].sum())

def add(x,y):
    return(1+x)*(1+y)

def yields(predict,returns): #output yileds
    real=returns.iloc[-len(predict):]
    if len(predict)==len(real):
        yields=[]
        for i in range(len(predict)):
            if int(predict.iloc[i])==1:
                yields.append(real.iloc[i]) 
            elif int(predict.iloc[i])==-1:
                yields.append(-real.iloc[i])
            else:
                yields.append(0)
#        accy=pd.DataFrame(reduce(add,yields))
            for i in range(len(yields)):
                if yields[i]<=-1:
                    return '爆仓'
                    
        yields=pd.DataFrame(yields)    
        return yields
        pring('wrong input')
def accy(yields)  : #ouput accumulated yields
    if type(yields)==str:
        return '爆仓'
    else:
        accy=[]
        for i in range(len(yields)):
                if i==0:
                    accy.append(1+yields.iloc[0])
                else:
                    accy.append((accy[i-1])*(1+yields.iloc[i]))
        return pd.DataFrame(accy)
def my_score(estimator,X_train,y_train,X_test,y_test):    
    model=estimator.fit(X_train,y_train)
    C=pd.DataFrame(confusion_matrix(y_test,pd.DataFrame(estimator.predict(X_test)) ,labels=[-1,0,1])) 
    if C[0].sum()!=0:
        TrueShortRate=C[0][0]/C[0].sum()
        FalseShortRate=C[0][2]/C[0].sum()
        CaughtShortRate=C[0][0]/C.iloc[0,:].sum()
    else:
        return -10000
    if C[2].sum()!=0:
        TrueLongRate=C[2][2]/C[2].sum()
        FalseLongRate=C[2][0]/C[2].sum()
        CaughtLongRate=C[2][2]/C.iloc[2,:].sum()
    else:
        return -10000
    if TrueShortRate-FalseShortRate<0 and TrueLongRate-FalseLongRate<0:
        return -10000
    score=(TrueShortRate-FalseShortRate)*(TrueLongRate-FalseLongRate)
    return score

In [7]:
# Read data
weekdata=pd.read_excel(r"D:\Enter\学习资料\Machine Learning in Finance\Project\weekly data-EN.xlsx",sheet_name='Sheet1')
weekdata=weekdata.iloc[:,1:].interpolate()  #Fill NaN

Input1=weekdata.pct_change().drop(index=[0,len(weekdata)-1]).reset_index(drop=1)      
Input2=weekdata.drop(index=[0,len(weekdata)-1]).reset_index(drop=1)   
Input=Input1.join(Input2,lsuffix='_pct').drop(columns=['PPI_I_yoy_pct','PPI_I_mom_pct'])
price=pd.read_excel(r"D:\Enter\学习资料\Machine Learning in Finance\Project\yields.xlsx",sheet_name='Sheet1').drop('Time',axis=1).drop(index=[0,1]).reset_index(drop=1)  
y=Dummy(price.iloc[:,1])
yr=pd.DataFrame(price.iloc[:,1])
print("Is Input there any NaN?",Input.isnull().values.any()) 
#Split Training and Test
X_train=Input.iloc[0:int(0.8*len(Input)),:]
X_test=Input.iloc[int(0.8*len(Input)):,:]
y_train=y.iloc[0:int(0.8*len(y)),:]
y_test=y.iloc[int(0.8*len(y)):,:]
yr_test=yr.iloc[int(0.8*len(y)):,:]
#X_train, X_test, y_train, y_test =\
#    train_test_split(Input, y, test_size=0.2, random_state=0, stratify=None)
tscv = TimeSeriesSplit(n_splits=5,max_train_size=50)
Input.head()

Is Input there any NaN? False


,Inv_Steel_pct,Inv_Iron_pct,GP_Rebar_pct,Cost_Rebar_pct,SP_Rebar_pct,SP_Iron_pct,FP_FS_pct,FB_Mn-Si_pct,FB_Coke_pct,SP_Coke_pct,...,M2,PPI,CA_REDI,CA_FAI,HCA,CSA,CHSA,PPI_I_yoy,PPI_I_mom,FP_Rebar.1
0,0.024660,0.004041,0.102218,0.038983,0.007880,0.003738,-0.018208,-0.010855,-0.193654,-0.000485,...,1519485.40,0.83,9377.0233,57522.5767,14951.9293,1.3,15153.7476,1.2,0.7,3039.0
1,0.006727,0.004293,0.035225,0.039732,0.063727,0.026071,-0.001455,-0.010507,0.020255,-0.005820,...,1530432.06,2.22,9412.4679,54118.9623,14140.9447,-0.5,15491.2065,3.3,1.5,3198.0
2,0.016677,-0.012378,-0.038519,0.079217,0.035875,-0.005445,-0.008012,-0.110665,-0.007657,-0.018537,...,1530432.06,2.22,9412.4679,54118.9623,14140.9447,-0.5,15491.2065,3.3,1.5,3251.0
3,0.029112,0.024795,-0.120141,-0.018459,0.034632,-0.034672,-0.031938,-0.011674,-0.059160,-0.014911,...,1530432.06,2.22,9412.4679,54118.9623,14140.9447,-0.5,15491.2065,3.3,1.5,3088.0
4,0.035524,0.000000,-0.398391,0.036036,-0.058050,-0.003781,-0.032992,-0.089933,-0.077764,-0.016145,...,1530432.06,2.22,9412.4679,54118.9623,14140.9447,-0.5,15491.2065,3.3,1.5,2923.0


In [8]:
y.head

<bound method NDFrame.head of      0
0    1
1    0
2   -1
3   -1
4    0
..  ..
248  0
249  0
250  0
251 -1
252 -1

[253 rows x 1 columns]>

In [9]:
yr.head()

,Return
0,0.052320
1,0.016573
2,-0.050138
3,-0.053433
4,0.012658


In [14]:
scaler = skpre.StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)
y_train=y_train*1
y_test=y_test*1
y_train=y_train.astype(int)
y_test=y_test.astype(int)   
pd.DataFrame(X_train_std)

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,0.308639,0.198692,0.305213,1.797514,0.334946,0.031710,-0.536027,-0.263113,-4.253643,-0.020784,...,-1.722676,-0.437733,-1.013452,0.253081,-0.353472,-0.323872,-0.217220,-0.316712,0.999226,-2.209761
1,0.035505,0.213459,0.062735,1.834078,3.156576,0.699887,-0.064282,-0.253724,0.420566,-0.213377,...,-1.660679,0.004696,-0.993258,0.014954,-0.357729,-0.324239,-0.126277,0.322742,2.402137,-1.761787
2,0.187051,-0.763254,-0.204179,3.759799,1.749355,-0.243040,-0.248920,-2.957171,-0.189352,-0.672471,...,-1.660679,0.004696,-0.993258,0.014954,-0.357729,-0.324239,-0.126277,0.322742,2.402137,-1.612463
3,0.376448,1.414695,-0.499605,-1.004007,1.686582,-1.117496,-0.922658,-0.285237,-1.314754,-0.541568,...,-1.660679,0.004696,-0.993258,0.014954,-0.357729,-0.324239,-0.126277,0.322742,2.402137,-2.071706
4,0.474105,-0.038048,-1.506717,1.653817,-2.996184,-0.193256,-0.952329,-2.397566,-1.721289,-0.586144,...,-1.660679,0.004696,-0.993258,0.014954,-0.357729,-0.324239,-0.126277,0.322742,2.402137,-2.536584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,-0.250980,0.619440,-2.658982,1.602028,-0.317923,-0.653448,0.026833,-0.602072,-0.912129,0.330256,...,1.773738,-1.328955,1.124573,-0.299808,-0.319334,-0.323954,-0.297840,-1.291119,0.297771,-0.598184
198,-0.887390,1.425135,-0.297143,-0.189926,-0.736918,0.065989,0.397249,-0.200602,0.888689,0.132798,...,1.773738,-1.328955,1.124573,-0.299808,-0.319334,-0.323954,-0.297840,-1.291119,0.297771,-0.609453
199,0.824095,0.680492,-2.522328,0.853779,0.391558,0.610593,0.677183,-0.023775,1.006212,0.035450,...,1.928058,-1.227101,2.207406,0.265214,-0.326512,-0.324097,0.707943,-1.321570,-0.052956,-0.522113
200,-0.888628,0.791315,8.229035,-0.239071,0.064024,-0.577614,0.544657,1.015291,0.209039,0.789929,...,1.928058,-1.227101,2.207406,0.265214,-0.326512,-0.324097,0.707943,-1.321570,-0.052956,-0.502390


In [15]:
#Select Features
lables=Input.columns[:]
forest = RandomForestClassifier(n_estimators=50,random_state=1)
forest.fit(X_train_std, y_train)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(X_train_std.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30,lables[indices[f]],importances[indices[f]]))
new = SelectFromModel(forest, threshold=0.00999,prefit=True)
X_train_std = pd.DataFrame(new.transform(X_train_std))
X_test_std   =pd.DataFrame(new.transform(X_test_std))

 1) SP_Rebar_pct                   0.038267
 2) Inv_Steel_pct                  0.033634
 3) FP_Rebar.1                     0.027421
 4) CN1YR_pct                      0.026681
 5) FB_Mn-Si_pct                   0.025906
 6) TV_Rebar_pct                   0.025661
 7)  GP_Rebar_pct                  0.023991
 8) SP_Coke                        0.022906
 9) FB_Coke_pct                    0.022774
10) FP_Rebar.1_pct                 0.021769
11) HP_Rebar_pct                   0.021163
12) FP_Iron_pct                    0.021154
13) FP_Rebar                       0.020406
14) US1YR_pct                      0.019332
15) FP_Rebar_pct                   0.019192
16) SP_Iron_pct                    0.019115
17) Cost_Rebar_pct                 0.019081
18) FB_Coke                        0.018385
19) Cost_Rebar                     0.018275
20) HP_Rebar                       0.017723
21) Inv_Iron_pct                   0.017626
22) SP_Coke_pct                    0.017531
23) FP_Iron                     

In [16]:
#LR model

Para_lr_C=[x/10 for x in range(1,400)]
lrT_score=-1  #store score
lrT_C=0
for lrC in Para_lr_C:
    lr= LogisticRegression(penalty='l2',
                       solver = 'lbfgs',C=lrC,class_weight={0:0.2, 1:0.4,-1:0.4},multi_class='multinomial'
                       )
    lr.fit(X_train_std, y_train)
    b=my_score(lr,X_train_std,y_train,X_test_std,y_test)
    if lrT_score<b:
        lrT_score=b
        lrT_C=lrC

lr= LogisticRegression(penalty='l2',
                       solver = 'lbfgs',C=lrT_C,class_weight={0:0.2, 1:0.4,-1:0.4},multi_class='multinomial'
                       )
lr.fit(X_train_std, y_train)
lr_pre=pd.DataFrame(lr.predict(X_test_std)) #  trading predicted by LR
y_y=y_test.reset_index(drop=1)
lr_confusion=pd.DataFrame(confusion_matrix(y_y,lr_pre,labels=[-1,0,1])) #LR's confusion matrix
lr_yields=yields(lr_pre,price.iloc[:,1]) #LR's yields
lr_accy=accy(lr_yields)
metrics(lr_confusion)      
my_score(lr,X_train_std,y_train,X_test_std,y_test)        
lr_CV_score=cross_val_score(LogisticRegression(penalty='l2',
                       solver = 'lbfgs',C=lrT_C,class_weight={0:0.2, 1:0.4,-1:0.4},multi_class='multinomial')
                           , X=Input, y=y, cv=tscv,scoring='f1_micro')    
print('LR best score',lrT_score)
print('LR best para C=',lrT_C)
print('LR return',lr_accy.iloc[-1,0]-1)
print('LR Cross validation score',lr_CV_score)

TrueShortRate= 0.375
FalseShortRate= 0.0
CaughtShortRate= 0.3
TrueLongRate= 0.5
FalseLongRate= 0.125
CaughtLongRate= 0.2222222222222222
LR best score 0.140625
LR best para C= 14.3
LR return 0.4620109862828252
LR Cross validation score [0.45238095 0.14285714 0.33333333 0.23809524 0.38095238]


In [17]:
#SVM model

Para_SVM_C=[x/10 for x in range(1,200,2)]
Para_SVM_gamma=[x/100 for x in range(1,2)]
SVMT_score=-1  #暂时储存score
SVMT_C=0
for SVMC in Para_SVM_C:
    for SVMgamma in Para_SVM_gamma:
        SVM=SVC(kernel='rbf', random_state=0, C=SVMC, gamma=SVMgamma,decision_function_shape='ovr') 
        SVM.fit(X_train_std, y_train)
        SVM_score=my_score(SVM,X_train_std,y_train,X_test_std,y_test)
        if SVMT_score<SVM_score:
            SVMT_score=SVM_score
            SVMT_C=SVMC
            SVMT_gamma=SVMgamma
SVM=SVC(kernel='rbf', random_state=0, C=SVMT_C, gamma=SVMgamma,decision_function_shape='ovr') 
SVM.fit(X_train_std, y_train)
SVM_pre=pd.DataFrame(SVM.predict(X_test_std)) 
y_y=y_test.reset_index(drop=1)
SVM_confusion=pd.DataFrame(confusion_matrix(y_y,SVM_pre,labels=[-1,0,1])) 
SVM_yields=yields(SVM_pre,price.iloc[:,1]) 
SVM_accy=accy(SVM_yields)
metrics(SVM_confusion)      
my_score(SVM,X_train_std,y_train,X_test_std,y_test)        
SVM_CV_score=cross_val_score(SVC(kernel='rbf', random_state=0, C=SVMC, gamma=SVMgamma,decision_function_shape='ovr') 
                           , X=Input, y=y, cv=tscv,scoring='f1_micro')    
print('SVM best score',SVMT_score)
print('SVM best para C=',SVMT_C)
print('SVM best para gamma=',SVMT_gamma)
print('SVM return',SVM_accy.iloc[-1,0]-1)
print('SVM Cross validation score',SVM_CV_score)

TrueShortRate= 0.25
FalseShortRate= 0.1875
CaughtShortRate= 0.4
TrueLongRate= 0.6666666666666666
FalseLongRate= 0.3333333333333333
CaughtLongRate= 0.1111111111111111
SVM best score 0.020833333333333332
SVM best para C= 10.9
SVM best para gamma= 0.01
SVM return 0.23718277473361327
SVM Cross validation score [0.33333333 0.66666667 0.54761905 0.57142857 0.47619048]


In [18]:
# DecisionTree model
Para_tree_depth=[x for x in range(1,47)]
treeT_score=-1  #
treeT_depth=0
for treedepth in Para_tree_depth:
    tree = DecisionTreeClassifier(criterion='gini',max_depth=treedepth)
    tree.fit(X_train_std, y_train)
    tree_score=my_score(tree,X_train_std,y_train,X_test_std,y_test)
    if treeT_score<tree_score:
        treeT_score=tree_score
        treeT_depth=treedepth
tree=DecisionTreeClassifier(criterion='gini',max_depth=treedepth)
tree.fit(X_train_std, y_train)
tree_pre=pd.DataFrame(tree.predict(X_test_std)) 
y_y=y_test.reset_index(drop=1)
tree_confusion=pd.DataFrame(confusion_matrix(y_y,tree_pre,labels=[-1,0,1])) 
tree_yields=yields(tree_pre,price.iloc[:,1]) #
tree_accy=accy(tree_yields)
metrics(tree_confusion)      
my_score(tree,X_train_std,y_train,X_test_std,y_test)        
tree_CV_score=cross_val_score(DecisionTreeClassifier(criterion='gini',max_depth=treedepth)
                          ,X=Input, y=y, cv=tscv,scoring='f1_micro')    
print('tree best score',treeT_score)
print('tree best para C=',treeT_depth)
print('tree return',tree_accy.iloc[-1,0]-1)
print('tree Cross validation score',tree_CV_score)

TrueShortRate= 0.15
FalseShortRate= 0.4
CaughtShortRate= 0.3
TrueLongRate= 0.23076923076923078
FalseLongRate= 0.15384615384615385
CaughtLongRate= 0.16666666666666666
tree best score 0.0
tree best para C= 6
tree return -0.17862521463422687
tree Cross validation score [0.47619048 0.28571429 0.5        0.38095238 0.30952381]


In [21]:
#Random forest model
Para_RF_n=[x for x in range(1,100)]
RFT_score=-1  
RFT_n=0
for RFn in Para_RF_n:
    RF = RandomForestClassifier(n_estimators=RFn,random_state=0, oob_score=1,criterion='gini')
    RF.fit(X_train_std, y_train)
    RF_score=my_score(RF,X_train_std,y_train,X_test_std,y_test)
    if RFT_score<RF_score:
        RFT_score=RF_score
        RFT_n=RFn
RF=RandomForestClassifier(n_estimators=RFn,random_state=0, oob_score=1,criterion='gini')
RF.fit(X_train_std, y_train)
RF_pre=pd.DataFrame(RF.predict(X_test_std)) 
y_y=y_test.reset_index(drop=1)
RF_confusion=pd.DataFrame(confusion_matrix(y_y,RF_pre,labels=[-1,0,1])) 
RF_yields=yields(RF_pre,price.iloc[:,1]) 
RF_accy=accy(RF_yields)
metrics(RF_confusion)      
my_score(RF,X_train_std,y_train,X_test_std,y_test)        
RF_CV_score=cross_val_score(RandomForestClassifier(n_estimators=RFn,random_state=0, oob_score=1,criterion='gini')
                          ,X=Input, y=y, cv=tscv,scoring='f1_micro')    
print('RF best score',RFT_score)
print('RF best para depth=',RFT_n)
print('RF return',RF_accy.iloc[-1,0]-1)
print('RF Cross validation score',RF_CV_score)

TrueShortRate= 0.25
FalseShortRate= 0.0
CaughtShortRate= 0.1
TrueLongRate= nan
FalseLongRate= nan
CaughtLongRate= 0.0
RF best score -0.0
RF best para depth= 12
RF return 0.07937434899425755
RF Cross validation score [0.47619048 0.28571429 0.5952381  0.42857143 0.33333333]


In [23]:
#Gradient Boosting Decision Tree model
Para_gbdt_n=[x for x in range(100,200)]
Para_gbdt_l=[x/10 for x in range(1,10)]
gbdtT_score=-1 
gbdtT_n=0
gbdtT_l=0
for gbdtn in Para_gbdt_n:
    for gbdtl in Para_gbdt_l:
        gbdt=GradientBoostingClassifier(n_estimators=gbdtn,learning_rate=gbdtl)
        gbdt.fit(X_train_std, y_train)
        gbdt_score=my_score(gbdt,X_train_std,y_train,X_test_std,y_test)
        if gbdtT_score<gbdt_score:
            gbdtT_score=gbdt_score
            gbdtT_n=gbdtn
            gbdtT_l=gbdtl
gbdt=GradientBoostingClassifier(n_estimators=gbdtT_n,learning_rate=gbdtT_l)
gbdt.fit(X_train_std, y_train)
gbdt_pre=pd.DataFrame(gbdt.predict(X_test_std))
y_y=y_test.reset_index(drop=1)
gbdt_confusion=pd.DataFrame(confusion_matrix(y_y,gbdt_pre,labels=[-1,0,1]))
gbdt_yields=yields(gbdt_pre,price.iloc[:,1]) 
gbdt_accy=accy(gbdt_yields)
metrics(gbdt_confusion)      
my_score(gbdt,X_train_std,y_train,X_test_std,y_test)        
gbdt_CV_score=cross_val_score(GradientBoostingClassifier(n_estimators=gbdtT_n,learning_rate=gbdtT_l)
                          ,X=Input, y=y, cv=tscv,scoring='f1_micro')    
print('gbdt best score',gbdtT_score)
print('gbdt best para n=',gbdtT_n)
print('gbdt best para learning rate=',gbdtT_l)
print('gbdt return',gbdt_accy.iloc[-1,0]-1)
print('gbdt Cross validation score',gbdt_CV_score)


TrueShortRate= 0.24324324324324326
FalseShortRate= 0.24324324324324326
CaughtShortRate= 0.9
TrueLongRate= 0.6
FalseLongRate= 0.2
CaughtLongRate= 0.16666666666666666
gbdt best score 0.017857142857142856
gbdt best para n= 101
gbdt best para learning rate= 0.8
gbdt return 0.11788099907814664
gbdt Cross validation score [0.47619048 0.23809524 0.42857143 0.23809524 0.33333333]
